In [1]:
import cv2
import os

%matplotlib inline

import dlib
from matplotlib import pyplot as plt
import numpy as np
import scipy.ndimage

from PIL import Image, ImageOps
import datetime


In [8]:
print(cv2.__version__)
print(dlib.__version__)

3.4.4
19.16.0


In [6]:
def scantree(path, ext):
    """Recursively yield DirEntry objects for given directory."""
    for entry in os.scandir(path):
        if entry.is_dir(follow_symlinks=False):
            yield from scantree(entry.path, ext)  # see below for Python 2.x
        elif entry.path.endswith(ext):
            yield entry

def get_img_file_names_in_dir(path, ext):
    return [entry.path for entry in scantree(path, ext)]


path = '/disks/data/datasets/selfie_project_datasets/tmp'

_, ds_dirname = os.path.split(path) 

min_dim_size = 300
min_size_to_resize = 100

files = get_img_file_names_in_dir(path, '.jpg')
print(len(files))
print(ds_dirname)

300134
tmp


In [26]:
# files_full = files[:]
# files = files[55127:]

In [ ]:
# get the frontal face detector
detector = dlib.get_frontal_face_detector()
        
# location of the standard landmark points for a face
predictor_path = 'shape_predictor_68_face_landmarks.dat';

# start capture
#video_capture = cv2.VideoCapture(0)

# build a predictor from the default shape predictor
predictor = dlib.shape_predictor(predictor_path);

then = datetime.datetime.now()
with open('faces.txt','a') as f:
    for ifile, file in enumerate(files):
        #count = 1;
        #start_time = datetime.now()
        
        img_dir, img_name = os.path.split(file)
        
        img_small_dir = img_dir.replace(ds_dirname, ds_dirname+'_small')
        os.makedirs(img_small_dir, exist_ok=True)
        
        img_small_faceinfo_dir = img_dir.replace( ds_dirname, ds_dirname + '_small_faceinfo')
        os.makedirs(img_small_faceinfo_dir, exist_ok=True)
        face_info_file = img_name.replace('.jpg','.info')
        
        try:
            frame = plt.imread(file)
        except:
            continue
            
        if(frame.shape==() or frame.shape[-1] >3):
            continue
            
        #frame = Image.open(file)
        orig_img_shape = frame.shape

        min_dim_img = np.min(orig_img_shape[:2])
        
        if(min_dim_img < 100):
            continue
        
        if (min_dim_img > min_size_to_resize):
            if(orig_img_shape[0] < orig_img_shape[1]):
                mul_fact = min_dim_size/orig_img_shape[0]
                img_shape = [min_dim_size, int(orig_img_shape[1]*mul_fact)]
            else:
                mul_fact = min_dim_size/orig_img_shape[1]
                img_shape = [int(orig_img_shape[0]*mul_fact), min_dim_size]

            frame = cv2.resize(frame, dsize = tuple(img_shape[1::-1]))
        
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        #print(orig_img_shape)
        #print(img_shape)
        
        #i = np.argmin(img_shape[:2])
        #img_shape  = np.multiply(np.array(img_shape), 1/shrink_fact).astype(np.int)

        
        #frame = cv2.cvtColor(np.array(ImageOps.fit(frame, img_shape, Image.ANTIALIAS), cv2.COLOR_RGB2BGR))
            #
        
        dets = detector(frame, 1)
        
        
        print("filename\tid\tface_count", file = f)
        print(file,ifile,len(dets), file = f)
        #print()
        
        if(len(dets) > 0):
            with open(os.path.join(img_small_faceinfo_dir,face_info_file),'w') as f_inf:
                # for each detected faces
                for i, d in enumerate(dets):

                    # retrieve the bounding box
                    x = d.left();
                    y = d.top();
                    x2 = d.right();
                    y2 = d.bottom();

                    # draw the bounding box
                    #cv2.rectangle(frame,(x,y),(x2,y2),(0,255,0),2)

                    # get the facial shape descriptor
                    shape = predictor(frame, d);

                    print(x,y,x2,y2, file = f_inf)

                    # for each of the sixty eight detected fiducial feature points
                    for pts in range(68):
                        print(shape.part(pts).x,shape.part(pts).y,  file = f_inf)
                        #cv2.circle(frame, (shape.part(pts).x, shape.part(pts).y), 2,(0,0,255))

        
        cv2.imwrite(os.path.join(img_small_dir,img_name), frame)
        
        if (ifile+1)%500==0:
            print("Completed",ifile)            
            now = datetime.datetime.now()
            print("elapsed time:",now - then)
            then = datetime.datetime.now()
    #plt.figure(figsize=(20,10))
    # show the annotated frame
    #plt.imshow(cv2.cvtColor(scipy.misc.imresize(frame,200), cv2.COLOR_BGR2RGB))
    #start_time = datetime.now()
   # print(frame.shape)

55127

233892

(500, 375)

(375, 500, 3)

(500, 375)